# Weather and Electricity Data Preprocessing Notebook


### Importing libraries

In [1]:

import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.feature_selection import mutual_info_regression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler




### Data formating

In [2]:

def format_climate(file_path):
    df = pd.read_csv(file_path)
    df.replace('-', pd.NA, inplace=True)
    #df.dropna(inplace=True)

    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    df = df.drop(columns='Station')

    df['Total Rainfall Mm'] = pd.to_numeric(df['Total Rainfall Mm'], errors='coerce')
    df['Mean Air Temperature in Celsius'] = pd.to_numeric(df['Mean Air Temperature in Celsius'], errors='coerce')
    df['Mean Vapour Pressure Hpa'] = pd.to_numeric(df['Mean Vapour Pressure Hpa'], errors='coerce')
    df['Mean Wind Speed M/Sec'] = pd.to_numeric(df['Mean Wind Speed M/Sec'], errors='coerce')
    df['Days Of Wind Gust >= 24 Knots in Day'] = pd.to_numeric(df['Days Of Wind Gust >= 24 Knots in Day'], errors='coerce')
    df['Standard Deviation Of Daily Mean Temperature in Celsius'] = pd.to_numeric(df['Standard Deviation Of Daily Mean Temperature in Celsius'], errors='coerce')

    df_grouped = df.groupby('Date').mean().reset_index()

    return df_grouped

In [3]:
def format_eletricity_data(file_path, business = False):
    df = pd.read_csv(file_path)
    
    df['Date'] = pd.to_datetime(df['Period start'], format='%d/%m/%y')
    #df.dropna(inplace=True)
    #df.reset_index(drop=True, inplace=True)


    if business:
        df = df.drop(columns=[
            'Regional demand (GWh)',
            'Proportion of regional demand (%)', 
            'National demand (GWh)',
            'Proportion of national demand (%)'])
        return df
    
    df = df.drop(columns=[
        'Est. Total demand (GWh)',
        'Est. proportion of regional demand',
        'Est. proportion of national demand', 'Average consumption (kWh)',
        'Average daily consumption (kWh)', '5th percentile (kWh)',
        '25th percentile (kWh)', '50th percentile (kWh)',
        '75th percentile (kWh)', '95th percentile (kWh)', 'Coverage %'])

    return df
    

In [4]:
def read_climate_list(file_list, file_path):
    data_list = []
    zone_names = ['UNI', 'LSI', 'USI', 'LNI', 'CNI']

    for i in file_list:
        data_list.append(format_climate(file_path+ i))
    
    for i in range(len(data_list)):
        data_list[i]['Region ID'] = zone_names[i]
        
    return data_list

In [5]:
def read_electricity_list(file_list, file_path):
    data_list = []
    data_list.append(format_eletricity_data(file_path+file_list[0], True))
    data_list.append(format_eletricity_data(file_path+file_list[1]))
    return data_list

In [6]:
def merge_data(climate_data, electricity_data):
    data = pd.merge(climate_data, electricity_data, on=['Date', 'Region ID'])
    data.drop(columns=[ 'Period end', 'Region ID', 'Region description', 'Region'], inplace=True)
        
    return data


### Read data files using helper functions above

In [7]:
ClimateDatafilelist = ['NewUpperNorthRegions.csv', 'NewLowerSouthRegions.csv', 'NewUpperSouthARegions.csv','NewLowerNorthRegions.csv', 'NewCentralNorthRegions.csv']
ClimatePath = '../ClimateDataProcessing/'
ElectricityDatafilelist = ['Business_demand_trends_20240901140306.csv', 'Residential_consumption_trends_20240901141207.csv']
ElectricityPath = '../ElectricityData/'

climate_data = read_climate_list(ClimateDatafilelist, ClimatePath)
electricity_data = read_electricity_list(ElectricityDatafilelist, ElectricityPath)

Remove the months not recorded

In [8]:
unobserved_rows = [175, 176, 177, 178, 179]
for i in range(len(climate_data)):
    climate_data[i].drop(unobserved_rows, inplace=True)

Find which data file has the missing value

In [9]:
for i in range(len(climate_data)):
    print(climate_data[i].isnull().sum()) 

Date                                                       0
Total Rainfall Mm                                          0
Mean Air Temperature in Celsius                            0
Mean Vapour Pressure Hpa                                   0
Mean Wind Speed M/Sec                                      0
Days Of Wind Gust >= 24 Knots in Day                       1
Standard Deviation Of Daily Mean Temperature in Celsius    0
Year                                                       0
Month                                                      0
Region ID                                                  0
dtype: int64
Date                                                       0
Total Rainfall Mm                                          0
Mean Air Temperature in Celsius                            0
Mean Vapour Pressure Hpa                                   0
Mean Wind Speed M/Sec                                      0
Days Of Wind Gust >= 24 Knots in Day                       0
Standard De

Check

In [10]:
climate_data[0][climate_data[0]['Days Of Wind Gust >= 24 Knots in Day'].isnull()]

,Date,Total Rainfall Mm,Mean Air Temperature in Celsius,Mean Vapour Pressure Hpa,Mean Wind Speed M/Sec,Days Of Wind Gust >= 24 Knots in Day,Standard Deviation Of Daily Mean Temperature in Celsius,Year,Month,Region ID
105,2018-10-01,59.65,14.55,13.2,3.4,NaN,1.45,2018.0,10.0,UNI


Fill by previous value

In [11]:
climate_data[0].ffill(inplace=True)

### Merge data files

Here we combine the dataframes to perform analysis by region easier.

In [12]:
electricity_Bus_res = pd.merge(electricity_data[0], electricity_data[1], how='inner')
electricity_Bus_res.head()


,Period start,Period end,Region ID,Region description,Selected business demand (GWh),Date,Region,Est. Total consumption (GWh)
0,1/01/10,31/01/10,CNI,Central North Island,502.788,2010-01-01,Central North Island,177.005
1,1/02/10,28/02/10,CNI,Central North Island,475.792,2010-02-01,Central North Island,162.780
2,1/03/10,31/03/10,CNI,Central North Island,539.230,2010-03-01,Central North Island,182.505
3,1/04/10,30/04/10,CNI,Central North Island,505.722,2010-04-01,Central North Island,188.663
4,1/05/10,31/05/10,CNI,Central North Island,519.263,2010-05-01,Central North Island,222.647


In [13]:
uni_climate_electricity = merge_data(climate_data[0], electricity_Bus_res)
cni_climate_electricity = merge_data(climate_data[4], electricity_Bus_res)
lni_climate_electricity = merge_data(climate_data[3], electricity_Bus_res)
usi_climate_electricity = merge_data(climate_data[2], electricity_Bus_res)
lsi_climate_electricity = merge_data(climate_data[1], electricity_Bus_res)



In [14]:
uni_climate_electricity.to_csv(f'Electricity-ResBusClimateUNI.csv', index=False) 
cni_climate_electricity.to_csv(f'Electricity-ResBusClimateCNI.csv', index=False)
lni_climate_electricity.to_csv(f'Electricity-ResBusClimateLNI.csv', index=False)
usi_climate_electricity.to_csv(f'Electricity-ResBusClimateUSI.csv', index=False)
lsi_climate_electricity.to_csv(f'Electricity-ResBusClimateLSI.csv', index=False)

# Transformations


# Feature Selection

def lasso_feature_selection(X, y, alpha):
    lasso = Lasso(alpha=alpha)
    lasso.fit(X, y)
    index = [i for i, coef in enumerate(lasso.coef_) if coef != 0]
    return lasso.coef_, X.columns[index]